# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [63]:
import pandas as pd
from pandas import DataFrame
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [64]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [65]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [66]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [67]:
#Printing first two rows, just to get columns name and to see data as well
file_path1 = 'event_datafile_new.csv'

# Open the CSV file
with open(file_path1, mode='r') as file:
    # Create a CSV reader object
    csv_reader1 = csv.reader(file)
    
    # Read and print the first two rows
    for i, row in enumerate(csv_reader1):
        print(row)
        if i == 1:  # Stop after the second row
            break

['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']
['Muse', 'Harper', 'M', '1', 'Barrett', '209.50159', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', '275', 'Supermassive Black Hole (Twilight Soundtrack Version)', '42']


In [68]:
# Load CSV file into a DataFrame
df = pd.read_csv('event_datafile_new.csv')

# Check the data types of each column
print(df.dtypes)

artist            object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
sessionId          int64
song              object
userId             int64
dtype: object


# Part II. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [69]:
#Make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [70]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [71]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

 
  
   
   


## Query 1 - Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

 Description
 
 1. Expected Output: We require to get artist name, song title and song length based upon sessionId=338 and itemInSession=4. So the following SQL is required to get this result.
 
 "SELECT artist, song, length from music_library_by_sessionid_and_itemInSession WHERE sessionId = '338' AND itemInSession='4'"
         
          
          
 2. Selecting of columns- We need to have the following columns
 sessionId , itemInSession , artist , song , length.
 
 3. Choice of Primary Keys: We should choose sessionID partition key as for each session id Item in session is unique. We can choose item in session  as clustering key. 
 Note- We can also choose the session id as clustering and Item in session as partition, this will not affect out result.
 
 4. Need to create a table and insert rows from above csv file: We will create table using SQL create if not existed to avoid any errors. the name of the table would be music_library_by_sessionid_and_itemInSession. 
 NOTE- We will make sure the order of create and insert would be same. In this case, make sure we have sessionId and itemInSession would be the first two column in this sequence only in create and insert columns, otherwise result would be affected.

In [72]:
## Creating a table name music_library_by_itemInSession
query = "CREATE TABLE IF NOT EXISTS music_library_by_sessionid_and_itemInSession"
query = query + """(sessionId int, itemInSession int, artist varchar, song varchar, length varchar,
primary key(sessionId, itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)             

#### Extracting data from file = 'event_datafile_new.csv' and using for loop putting all the values.

In [73]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_by_sessionid_and_itemInSession(sessionId,itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], line[5]))

### The artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [74]:
query = """SELECT artist, song, length from music_library_by_sessionid_and_itemInSession
        WHERE sessionId =338 AND itemInSession=4 """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist,"|", row.song,"|", row.length)

Faithless | Music Matters (Mark Knight Dub) | 495.3073


## QUERY 2 - Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

 Description
 
 1. Expected Output: We require to get artist name, song title and song length based upon userId=10 and sessionId=182, sorted by itemInSession. So the following SQL is required to get this result.
 
 "SELECT artist, song, firstname, lastname from music_library_userid_sessionid
WHERE userid='10' AND  sessionid = '182'"
        
         
 2. Selecting of columns- We need to have the following columns
 userid, sessionid, itemInSession, artist, song, firstName, lastName
 
 3. Choice of Primary Keys: We should choose userid as partition key. For first clustering column it has to be sessionId to as we need to filter data accordint to it and this will  make our key unique as well. The second clustering column would be itemInSession as we have to sort our results based upon it.
 
 
 
 4. Need to create a table and insert rows from above csv file: We will create table using SQL create if not existed to avoid any errors. the name of the table would be music_library_userid_sessionid. 
 NOTE- We will make sure the order of create and insert would be same. In this case, make sure we have userid, sessionid and iteminSession  would be the first three columns in this sequence only in create and insert columns, otherwise result would be affected.

In [75]:
## Creating a table name music_library_userid_sessionid
query = "CREATE TABLE IF NOT EXISTS music_library_userid_sessionid "
query = query + """(userid int, sessionid int, itemInSession int, artist varchar, song varchar,
firstName varchar, lastName varchar, primary key(userid,sessionid,itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)          

#### Extracting data from file = 'event_datafile_new.csv' and using for loop putting all the values.

In [76]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = """INSERT INTO music_library_userid_sessionid (userid, sessionid, itemInSession,  
        artist, song, firstName, lastName)"""
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

### Name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 

In [77]:
query = """SELECT iteminsession, artist, song, firstname, lastname from music_library_userid_sessionid
WHERE userid=10 AND  sessionid =182 """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Print the results
for row in rows:
    print(row.artist, "|", row.song, "|", row.firstname, "|", row.lastname)

Down To The Bone | Keep On Keepin' On | Sylvie | Cruz
Three Drives | Greece 2000 | Sylvie | Cruz
Sebastien Tellier | Kilometer | Sylvie | Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie | Cruz


## QUERY 3 - Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

  Description
 
 1. Expected Output: We need to get first and last name of each users who are istening to one title song "All Hands Against His Own". So for sure we need to make query like this:
 "SELECT firstname, lastname from music_library_songtitle WHERE song='All Hands Against His Own'"
 
 2. Selecting of columns- We need to have the following columns
song, userid, firstName, lastName
 
 2. Choice of Primary Keys: For partition key we will definalty go with Song name. Now this is possible that there are many people with same first and last name and listening to same song. So we will add userid to clustering columns to make our primary key unique.
 
 3. Need to create a table:We will create table using SQL create if not existed to avoid any errors. the name of the table would be music_library_songtitle.
 NOTE- We will make sure the order of create and insert would be same

In [78]:
query = "CREATE TABLE IF NOT EXISTS music_library_songtitle "
query = query + """(song varchar, userid int,firstName varchar, lastName varchar,
primary key(song, userid))"""
try:
    session.execute(query)
except Exception as e:
    print(e)    

In [79]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_songtitle(song, userid, firstName, lastName)"
        query = query + " VALUES ( %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

### Every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own

In [80]:
query = "SELECT firstname, lastname from music_library_songtitle WHERE song='All Hands Against His Own'  "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname,"|", row.lastname)

Jacqueline | Lynch
Tegan | Levine
Sara | Johnson


### Drop the tables before closing out the sessions

In [81]:
query1 = "drop table music_library_by_sessionid_and_itemInSession"
try:
    rows1 = session.execute(query1)
except Exception as e:
    print(e)

query2 = "drop table music_library_userid_sessionid"
try:
    rows2 = session.execute(query2)
except Exception as e:
    print(e)

query3 = "drop table music_library_songtitle"
try:
    rows3 = session.execute(query3)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [82]:
session.shutdown()
cluster.shutdown()